In [ ]:
from datasets import load_dataset

In [ ]:
# get entire dataset
dataset = load_dataset("midas/kpcrowd", "raw")
dataset

In [ ]:
# sample from the test split
test_dataset  = dataset["test"]
test_dataset

In [ ]:
row=test_dataset[1]
row['extractive_keyphrases']

In [ ]:
import json
import boto3
from uuid import uuid4
from nltk.tokenize.treebank import TreebankWordDetokenizer

universe_bucket=""
documents_prefix = f"midas/kpcrowd/documents"
keyphrases_prefix = f"midas/kpcrowd/keyphrases"

s3_client = boto3.client("s3")
for row in test_dataset:
    text = TreebankWordDetokenizer().detokenize(row['document'])
    id = row['id']

    json_obj = {
        "document_id": id,
        "keyphrases": row['extractive_keyphrases']
    }
    file_name = str(uuid4())
    key = f"{documents_prefix}/id={id}/{file_name}.txt"
    s3_client.put_object( Bucket=universe_bucket, Key=key, Body=text)

    key = f"{keyphrases_prefix}/id={id}/{file_name}.json"
    s3_client.put_object( Bucket=universe_bucket, Key=key, Body=json.dumps(json_obj))